# LGBM Prediction



In [1]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/RecSys2024/'
# 3.6.0以降だとLightGBMTunerが動かない
#!pip3 install optuna==3.5.0
!pip3 install polars lightgbm pyarrow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#DEBUG_MODE = True
DEBUG_MODE = False
TRAIN_FRAC = 1.0 # train dataのnegative sampleをどれくらいsamplingするか
#TRAIN_FRAC = 0.8 # train dataのnegative sampleをどれくらいsamplingするか
#TRAIN_FRAC = 0.5 # train dataのnegative sampleをどれくらいsamplingするか
VALID_FRAC = 0.25 # validatoin dataをどれくらいsamplingするか
#lgbm_valid_frac1.0.pkl

train_type = 'train'
#train_type = 'valid' # use validation data as training for final su

In [3]:
# ==================================================== # Library # ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import polars as pl
import joblib
import pyarrow
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, log_loss
import lightgbm as lgb
print(lgb.__version__)

4.3.0


# Reading Input File

In [4]:
N_split = 2 # N個に分割して推論
out_path = base_path + 'feature_output'
model_path = base_path + 'models'
TRAIN_FRAC = '1.0'
bins = [600, 3600, 3600 * 6]

# Feature Engineering

In [5]:
def read_files(i_chunk, N_split=2):
    impression = pl.read_parquet(f"{out_path}/test_impression.parquet").sort(["impression_id", "user_id"])
    impression_article = pl.read_parquet(f"{out_path}/test_impression_article.parquet").sort(["impression_id", "user_id"])
    user = pl.read_parquet(f"{out_path}/test_user.parquet")
    user_article = pl.read_parquet(f"{out_path}/test_user_article.parquet")
    article = pl.read_parquet(f"{out_path}/test_article.parquet")

    rows_per_chunk = len(impression) // N_split
    chunks = [impression.slice(i * rows_per_chunk, rows_per_chunk) for i in range(N_split)]
    impression = chunks[i_chunk]
    del chunks, rows_per_chunk
    gc.collect()
    return impression_article, impression, user, user_article, article

def add_pop_count(df, out_col_name, bins):

    for bin in bins:
        df = df.with_columns(
            (df["impression_time"] / bin).cast(pl.Int64).alias(f"time_bin_{bin}")
        )
        grouped_df = df.group_by([f"time_bin_{bin}", "article_id_inview"]).agg(
            pl.count().cast(pl.Int32).alias(out_col_name + str(bin))
        )
        df = df.join(
            grouped_df,
            on=[f"time_bin_{bin}", "article_id_inview"],
            how="left"
        )
        df = df.drop(f"time_bin_{bin}")
    return df


def join_features(impression_article, impression, user, user_article, article, bins):
    print('join features....')
    df = impression_article.join(impression, on=["impression_id", "user_id"], how="inner")
    del impression_article
    gc.collect()
    df = reduce_mem_usage(df)

    print('add view_cnt....')
    df = add_pop_count(df,"view_cnt", bins)
    df = reduce_mem_usage(df)

    print('add user features....')
    df = df.join(user, on="user_id", how="left")
    del user
    gc.collect()
    df = reduce_mem_usage(df)

    print('add article features....')
    df = df.rename({
        "article_id_inview": "to_article_id"
    })
    df = df.join(
        article.rename({col: f"to_{col}" for col in article.columns}),
        on="to_article_id",
        how="left"
    )
    del article
    gc.collect()
    df = reduce_mem_usage(df)

    print('add user article features....')
    df = df.join(
        user_article.rename({col: f"to_{col}" for col in user_article.columns if col != 'user_id'}),
        on=["user_id", "to_article_id"],
        how="left"
    )
    del user_article
    gc.collect()
    df = reduce_mem_usage(df)

    print('cast features....')
    for bin in bins:
        df = df.with_columns(
            pl.col(f"view_cnt{bin}").fill_null(0).cast(pl.Int32)
        )
    df = reduce_mem_usage(df)
    return df


def generate_unixtime_features(df, unixtime_list):

    for col_name in unixtime_list:
        df = df.with_columns(
            (pl.col("impression_time") - pl.col(col_name)).alias(f"{col_name}_diff").cast(pl.Int64)
        )
        df = df.drop(col_name)
    df = df.drop('impression_time')
    df = reduce_mem_usage(df)
    return df


def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data
    #return df_data, cat_cols


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage in Polars.
    """

    # Initialize an empty list to store optimized columns
    optimized_columns = []

    for col in df.columns:
        col_data = df[col]
        col_type = col_data.dtype

        if col_type in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt32]:
            col_data = col_data.fill_null(0)
            c_min = col_data.min()
            c_max = col_data.max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                optimized_columns.append(col_data.cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                optimized_columns.append(col_data.cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                optimized_columns.append(col_data.cast(pl.Int32))
            else:
                optimized_columns.append(col_data.cast(pl.Int64))
        elif col_type in [pl.Float32, pl.Float64]:
            c_min = col_data.min()
            c_max = col_data.max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                optimized_columns.append(col_data.cast(pl.Float32))
            else:
                optimized_columns.append(col_data.cast(pl.Float64))
        else:
            optimized_columns.append(col_data)
    # Create a new DataFrame with optimized columns
    optimized_df = pl.DataFrame({col: optimized_columns[i] for i, col in enumerate(df.columns)})

    return optimized_df


def cast_float_to_int(df, columns):
    for col in columns:
        df = df.with_columns(pl.col(col).cast(pl.Int64))
    return df


def negative_sampling(df, frac=0.5, seed=42):
    negative_examples = df.filter(pl.col("clicked") == False)
    positive_examples = df.filter(pl.col("clicked") == True)
    reduced_negatives = negative_examples.sample(fraction = frac, seed=seed)

    return pl.concat([reduced_negatives, positive_examples]).sort("impression_id")

In [6]:
# is_beyond_accuracyはtest dataだけ入ってるので削る
IGNORE_COL_ID = ["impression_id", "session_id", "is_beyond_accuracy", "user_id", "to_article_id", "view_pos"]

model = np.load(f'{model_path}/lgbm_{train_type}_frac{TRAIN_FRAC}.pkl', allow_pickle=True)
model

In [ ]:
sub_list = []

for i in range(N_split):
    print(f'chunk {i}')
    test_impression_article, test_impression, test_user, test_user_article, test_article = read_files(i_chunk = i, N_split=N_split)
    # memory節約, intだけどfloatになってるはいったんint64にしてから変換
    float_int_cols = ['read_time', 'scroll_percentage']
    test_impression = cast_float_to_int(test_impression, float_int_cols)
    test_impression = reduce_mem_usage(test_impression)
    test_impression_article = reduce_mem_usage(test_impression_article)

    float_int_cols = ['u_read_time_last', 'u_scroll_percentage_last', 'u_impression_time_mean']
    test_user = cast_float_to_int(test_user, float_int_cols)
    test_user = reduce_mem_usage(test_user)

    float_int_cols = ['ua_read_time_last', 'ua_scroll_percentage_last']
    test_user_article = cast_float_to_int(test_user_article, float_int_cols)
    test_user_article = reduce_mem_usage(test_user_article)

    float_int_cols = ['a_total_read_time']
    test_article = cast_float_to_int(test_article, float_int_cols)
    test_article = reduce_mem_usage(test_article)

    test = join_features(test_impression_article, test_impression, test_user, test_user_article, test_article, bins)
    del test_impression_article, test_impression, test_user, test_user_article, test_article
    gc.collect()

    # unixtimeのfeatureをdiffにする
    unixtime_list = ['u_impression_time_last', 'u_impression_time_mean', 'to_ua_impression_time_mean', 'to_ua_impression_time_last', 'to_a_published_time', 'to_a_last_modified_time']
    test = generate_unixtime_features(test, unixtime_list)

    display(test.shape)
    display(test.head())
    test = to_pandas(test)

    query_list_test = test['impression_id'].value_counts()
    query_list_test = query_list_test.sort_index()
    print('remove id from features....')
    test_id = test[['impression_id', 'user_id', 'to_article_id', 'view_pos']]
    test = test.drop(IGNORE_COL_ID, axis=1)
    pred = model.predict(test)

    test_id = test_id.reset_index(drop=True)
    pred = pd.Series(pred, name='score')
    sub = pd.concat([test_id, pred], axis=1)
    sub_list.append(sub)

    del test_id, pred, sub
    gc.collect()


chunk 0
join features....
add view_cnt....
add user features....
add article features....
add user article features....


In [ ]:
sub = pd.concat(sub_list, ignore_index=True)
del sub_list
gc.collect()

## Output and Save

In [ ]:
sub = pl.from_pandas(sub)
sub = sub.sort("impression_id", "user_id", "score", descending=[False, False, True])
sub.write_parquet(f'{base_path}/output/score_lgbm_{train_type}_frac{TRAIN_FRAC}.parquet')
sub

In [ ]:
out = sub.groupby(["impression_id", "user_id"]).agg(pl.col("view_pos"))
id = pl.read_parquet(f"{out_path}/test_impression.parquet").select([
    pl.col("impression_id").cast(pl.Int32),
    pl.col("user_id").cast(pl.Int32)
])
out = id.join(out, on=["impression_id", "user_id"], how='left')
out

In [ ]:
# check beyond accuracy impression
out.filter(pl.col('impression_id') == 0)

In [ ]:
with open(f'{base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt', 'w') as f:
    for row in out.to_dicts():
        # 一度strにしないとlistの","の間に半角スペースが入ってしまう
        view_pos_str = ','.join(map(str, row['view_pos']))
        line = f"{row['impression_id']} [{view_pos_str}]\n"
        #line = f"{row['impression_id']} {row['view_pos']}\n"
        f.write(line)

### Check output

In [ ]:
!head {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt

In [ ]:
!tail {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt

In [ ]:
!wc -l {base_path}/output/pred_lgbm_{train_type}_frac{TRAIN_FRAC}.txt